In [1]:
from __future__ import print_function
%config IPCompleter.greedy=True

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

import pandas as pd
import nltk.data
import re, string, unicodedata
import nltk
import matplotlib.pyplot as plt
import numpy as np
import ast

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
C:\Users\ralvarez\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df = pd.read_csv("../Data/Flujo1.csv", sep=";")
df['Respuesta'] = df['Respuesta'].apply(ast.literal_eval)

In [ ]:
model = Word2Vec.load('../modelWord2vec.bin')

In [22]:
def text_to_vectors(array_text):
    array_vectors = []
    for word in array_text:
        array_vectors.append(model.wv['word'])
    return np.array(array_vectors)
data = pd.DataFrame()
data['_vectors'] = df["Respuesta"].apply(text_to_vectors)

In [4]:
def get_mean_vector(vectors):
    return np.mean(vectors, axis=0)
data['_vectors'] = data["_vectors"].apply(get_mean_vector)

In [5]:
list_dimensions = df.Dimension.unique().tolist()
def enumerate_dimensions(dimension):
    return list_dimensions.index(dimension)
data['_dimension'] = df.Dimension.apply(enumerate_dimensions)

## Prueba con Keras Preprocessing .... 

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
def make_phrases(tokens):
    string = ""
    for token in tokens:
        string = string + token + " "
    return string
# make_phrases(df.Respuesta[0])
df.Respuesta = df.Respuesta.apply(make_phrases)

In [5]:
list_dimensions = df.Dimension.unique().tolist()
def enumerate_dimensions(dimension):
    return list_dimensions.index(dimension)
df.Dimension = df.Dimension.apply(enumerate_dimensions)

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.Respuesta.values)
sequences = tokenizer.texts_to_sequences(df.Respuesta.values)
vocab = len(tokenizer.word_index) + 1

In [7]:
sequences

[[363, 47, 935, 457, 1809],
 [2351, 175, 12, 3, 48, 432, 162, 597, 480],
 [1810, 30, 5, 27, 188, 70, 14080],
 [662, 3, 248, 190, 215, 3, 248, 590, 4454],
 [156, 19, 332, 56, 4821, 686, 125, 315, 2422, 45],
 [156, 19, 47, 244],
 [179, 1086, 112, 349],
 [1014, 949, 950, 143, 372],
 [3, 237, 112, 349],
 [3, 362, 1086, 112, 349, 17, 163, 2606, 3466],
 [3, 502, 45, 244],
 [204, 414, 790, 478, 949],
 [480, 6989, 2483, 112, 349, 1654, 372],
 [662, 105, 66, 1655, 1687, 126, 176],
 [52, 504, 1, 333, 357, 772, 4822],
 [772, 713, 10467, 5060, 584, 2134, 1050, 332, 1103, 10468],
 [333, 45, 244, 298, 10, 480, 2352, 7726],
 [102, 368, 112, 349, 713, 2030, 14081, 974, 14082],
 [256, 535, 958, 1370, 68],
 [357, 330, 2313, 469, 1654, 26, 2031],
 [30, 330, 41, 28, 1, 3139, 748, 216, 112, 349],
 [30, 47, 45, 46, 144],
 [1563, 144, 6990, 1563, 144, 3550, 2717],
 [249, 2, 5998, 550, 568],
 [94, 24, 226, 569, 14083],
 [462, 3, 11, 1855],
 [56, 112, 349, 125],
 [396, 2810, 3, 248, 249],
 [171, 857, 1014, 190

In [9]:
X = pad_sequences(sequences, maxlen=10, padding='post')

In [10]:
y = df.Dimension.values

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten
model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=10, input_length=10))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

W0923 01:12:31.093281 16288 deprecation.py:506] From C:\Users\ralvarez\Anaconda\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0923 01:12:31.132297 16288 deprecation.py:506] From C:\Users\ralvarez\Anaconda\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0923 01:12:31.190546 16288 deprecation.py:323] From C:\Users\ralvarez\Anaconda\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops

In [64]:
from tensorflow.keras.callbacks import EarlyStopping
early = EarlyStopping(monitor='val_loss', patience=10)

In [65]:
history = model.fit(X_train, y_train, batch_size=128, validation_data=(X_test, y_test), callbacks=[early])

Train on 119956 samples, validate on 39986 samples
119956/119956 [==============================] - 4s 30us/sample - loss: -440.6986 - acc: 0.0681 - val_loss: -1329.6993 - val_acc: 0.0664


In [66]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 10)            308780    
_________________________________________________________________
flatten_1 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 308,881
Trainable params: 308,881
Non-trainable params: 0
_________________________________________________________________


In [67]:
history.history

{'loss': [-440.6985732008185],
 'acc': [0.06809163],
 'val_loss': [-1329.6993021532146],
 'val_acc': [0.06637323]}

In [68]:
def predict(model=model, texto='realmente tiene muchos problemas, como gobierno y temas de finanzas'):
    x = np.array([texto])
    seq = tokenizer.texts_to_sequences(x)
    padded = pad_sequences(seq, maxlen=10, padding='post')
    pred = model.predict_classes(padded)
    return pred

In [69]:
predict()

array([[1]])

## LSTM

In [14]:
from tensorflow.keras.layers import LSTM, Dropout
model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=10, input_length=10))
model.add(LSTM(10, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=10, batch_size=512, validation_data=(X_test, y_test))

Train on 119956 samples, validate on 39986 samples
Epoch 1/10
119956/119956 [==============================] - 9s 77us/sample - loss: -3598195514.1325 - acc: 0.0679 - val_loss: -71793612390.3590 - val_acc: 0.0672
Epoch 2/10
119956/119956 [==============================] - 6s 53us/sample - loss: -623379425110738.7500 - acc: 0.0679 - val_loss: -5547264579771106.0000 - val_acc: 0.0672
Epoch 3/10
119956/119956 [==============================] - 8s 65us/sample - loss: -521552944368277248.0000 - acc: 0.0679 - val_loss: -2886669301739348480.0000 - val_acc: 0.0672
Epoch 4/10
119956/119956 [==============================] - 7s 57us/sample - loss: -47296521307760549888.0000 - acc: 0.0679 - val_loss: -186500390657319600128.0000 - val_acc: 0.0672
Epoch 5/10
119956/119956 [==============================] - 9s 79us/sample - loss: nan - acc: 0.0353 - val_loss: nan - val_acc: 0.0127
Epoch 6/10
119956/119956 [==============================] - 7s 60us/sample - loss: nan - acc: 0.0135 - val_loss: nan - v

In [15]:
model